In [138]:
!pip install unidecode -q
!pip install contractions
!pip install nltk
!pip install gensim

In [122]:
import pandas as pd
import numpy as np
import logging
import re
import multiprocessing
from time import time 
from nltk.corpus import stopwords
import contractions
from unidecode import unidecode
from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
df_ = pd.read_csv('musk.csv', index_col=0)
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)
df_

,type,author,text,created
0,Comment,UNCwesRPh,I had been on twitter prior to the musk takeov...,2023-01-30 18:49:31+00:00
2,Comment,wgp3,That article does not say what you imply at al...,2023-01-30 18:49:28+00:00
3,Comment,HighAndDrunk,The OG musk duck lives on my wall.,2023-01-30 18:48:43+00:00
4,Comment,Louismaxwell23,How dare he speak that way to the great and po...,2023-01-30 18:48:26+00:00
5,Comment,Copykill,Can’t wait to finally have an excuse not to sh...,2023-01-30 18:48:10+00:00
...,...,...,...,...
85215,Comment,Emble12,"Like brain dead piranhas lmao, anything barely...",2023-05-01 07:09:53+00:00
85221,Comment,Pizza_in_Space,What am I lying about? What's my agenda? Pleas...,2023-05-01 07:08:43+00:00
85223,Comment,Da1realBigA,Hard disagree. I don't think he's a parody of ...,2023-05-01 07:07:36+00:00
85224,Comment,Viperions,Yeah. I think too many things are lining up ri...,2023-05-01 07:07:15+00:00


In [100]:
df_['type'].unique()

array(['Comment', '2023-04-30 15:47:51+00:00'], dtype=object)

In [101]:
# Remove column with only one value
df_.drop(columns=['type'])

,author,text,created
0,UNCwesRPh,I had been on twitter prior to the musk takeov...,2023-01-30 18:49:31+00:00
2,wgp3,That article does not say what you imply at al...,2023-01-30 18:49:28+00:00
3,HighAndDrunk,The OG musk duck lives on my wall.,2023-01-30 18:48:43+00:00
4,Louismaxwell23,How dare he speak that way to the great and po...,2023-01-30 18:48:26+00:00
5,Copykill,Can’t wait to finally have an excuse not to sh...,2023-01-30 18:48:10+00:00
...,...,...,...
85215,Emble12,"Like brain dead piranhas lmao, anything barely...",2023-05-01 07:09:53+00:00
85221,Pizza_in_Space,What am I lying about? What's my agenda? Pleas...,2023-05-01 07:08:43+00:00
85223,Da1realBigA,Hard disagree. I don't think he's a parody of ...,2023-05-01 07:07:36+00:00
85224,Viperions,Yeah. I think too many things are lining up ri...,2023-05-01 07:07:15+00:00


In [102]:
def remove_duplicates_and_na(df:pd.DataFrame) -> pd.DataFrame:
    '''Remove duplicates and NA values'''
    df.dropna(inplace=True)
    df.drop_duplicates(inplace=True)
    return df

In [103]:
def lower(text:str) -> str:
    '''Lower all characters'''
    return text.lower()

In [104]:
def replace_contractions(text:str) -> str:
    '''Replace contractions in string of text'''
    return contractions.fix(text)

In [105]:
def remove_url(text:str) -> str:
   '''Remove URLs'''
   return re.sub(r"((www.[^\s]+)|(https?://[^\s]+))", " ", text)

In [106]:
def replace_to_unicode(text:str) -> str:
    '''Replace special characters to unicode standard'''
    return unidecode(text)

In [107]:
def text_to_word_list(text:str) -> str:
    ''' Pre process and convert texts to a list of words '''
    text = str(text)

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"'s ", " is ", text)
    text = re.sub(r"'ve ", " have ", text)
    text = re.sub(r"n't ", " not ", text)
    text = re.sub(r"'re ", " are ", text)
    text = re.sub(r"'d ", " would ", text)
    text = re.sub(r"'ll ", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub(r"/r/", " subreddit ", text)
    text = re.sub(r"0 0", "00", text)
    text = text.split()

    return text

In [108]:
df = df_.copy()
df = remove_duplicates_and_na(df)
df['text'] = df['text'].apply(lambda x: lower(x))
df['text'] = df['text'].apply(lambda x: remove_url(x))
df['text'] = df['text'].apply(lambda x: replace_contractions(x))
df['text'] = df['text'].apply(lambda x: replace_to_unicode(x))
df['text'] = df['text'].apply(lambda x: text_to_word_list(x))

In [109]:
df

,type,author,text,created
0,Comment,UNCwesRPh,"[i, had, been, on, twitter, prior, to, the, mu...",2023-01-30 18:49:31+00:00
2,Comment,wgp3,"[that, article, does, not, say, what, you, imp...",2023-01-30 18:49:28+00:00
3,Comment,HighAndDrunk,"[the, og, musk, duck, lives, on, my, wall]",2023-01-30 18:48:43+00:00
4,Comment,Louismaxwell23,"[how, dare, he, speak, that, way, to, the, gre...",2023-01-30 18:48:26+00:00
5,Comment,Copykill,"[cannot, wait, to, finally, have, an, excuse, ...",2023-01-30 18:48:10+00:00
...,...,...,...,...
85215,Comment,Emble12,"[like, brain, dead, piranhas, lmao, anything, ...",2023-05-01 07:09:53+00:00
85221,Comment,Pizza_in_Space,"[what, am, i, lying, about, what, is, my, agen...",2023-05-01 07:08:43+00:00
85223,Comment,Da1realBigA,"[hard, disagree, i, do, not, think, he, is, a,...",2023-05-01 07:07:36+00:00
85224,Comment,Viperions,"[yeah, i, think, too, many, things, are, linin...",2023-05-01 07:07:15+00:00


In [110]:
def remove_stop_words(df:pd.DataFrame) -> pd.DataFrame:
    '''Remove stop words'''
    stops = set(stopwords.words('english'))
    df['text'] = df['text'].apply(lambda x: [item for item in x if item not in stops])
    return df

In [111]:

df = remove_stop_words(df)
df = df[df.text.str.len() > 1]

In [112]:
sent = [row for row in df.text]
phrases = Phrases(sent, min_count=1, progress_per=50000)
bigram = Phraser(phrases)
sentences = bigram[sent]
sentences[1]

INFO - 15:45:58: collecting all words and their counts
INFO - 15:45:58: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 15:46:00: PROGRESS: at sentence #50000, processed 1970668 words and 1235972 word types
INFO - 15:46:00: collected 1463211 token types (unigram + bigrams) from a corpus of 2431210 words and 61282 sentences
INFO - 15:46:00: merged Phrases<1463211 vocab, min_count=1, threshold=10.0, max_vocab_size=40000000>
INFO - 15:46:00: Phrases lifecycle event {'msg': 'built Phrases<1463211 vocab, min_count=1, threshold=10.0, max_vocab_size=40000000> in 2.00s', 'datetime': '2023-05-20T15:46:00.847714', 'gensim': '4.3.1', 'python': '3.9.9 (v3.9.9:ccb0e6a345, Nov 15 2021, 13:06:05) \n[Clang 13.0.0 (clang-1300.0.29.3)]', 'platform': 'macOS-13.3-arm64-arm-64bit', 'event': 'created'}
INFO - 15:46:00: exporting phrases from Phrases<1463211 vocab, min_count=1, threshold=10.0, max_vocab_size=40000000>
INFO - 15:46:02: FrozenPhrases lifecycle event {'msg': 'exported Frozen

['article',
 'say',
 'imply',
 'states',
 'feature',
 'turns',
 'crashes',
 'tesla',
 'admits',
 'default_behavior',
 'never',
 'tried_hide',
 'hence',
 'whole',
 'counting',
 'crash',
 'disengaged',
 'within',
 '5_seconds',
 'crash',
 'article',
 'say',
 'brings',
 'question',
 'times',
 'musk',
 'said',
 'feature',
 'give',
 'proof',
 'times',
 'musk',
 'said',
 'fact',
 'referring',
 'crashes',
 'turned',
 'right',
 'crash',
 'speculation',
 'best']

In [113]:
df.shape

(61282, 4)

In [114]:
df.to_csv('musk_clean.csv')

In [115]:
w2v_model = Word2Vec(min_count=3,
                     window=4,
                     vector_size=300,
                     sample=1e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=multiprocessing.cpu_count()-1)

start = time()

w2v_model.build_vocab(sentences, progress_per=50000)

print('Time to build vocab: {} mins'.format(round((time() - start) / 60, 2)))

INFO - 15:46:03: Word2Vec lifecycle event {'params': 'Word2Vec<vocab=0, vector_size=300, alpha=0.03>', 'datetime': '2023-05-20T15:46:03.189513', 'gensim': '4.3.1', 'python': '3.9.9 (v3.9.9:ccb0e6a345, Nov 15 2021, 13:06:05) \n[Clang 13.0.0 (clang-1300.0.29.3)]', 'platform': 'macOS-13.3-arm64-arm-64bit', 'event': 'created'}
INFO - 15:46:03: collecting all words and their counts
INFO - 15:46:03: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 15:46:04: PROGRESS: at sentence #50000, processed 1628170 words, keeping 175838 word types
INFO - 15:46:04: collected 193416 word types from a corpus of 2009180 raw words and 61282 sentences
INFO - 15:46:04: Creating a fresh vocabulary
INFO - 15:46:04: Word2Vec lifecycle event {'msg': 'effective_min_count=3 retains 70944 unique words (36.68% of original 193416, drops 122472)', 'datetime': '2023-05-20T15:46:04.886599', 'gensim': '4.3.1', 'python': '3.9.9 (v3.9.9:ccb0e6a345, Nov 15 2021, 13:06:05) \n[Clang 13.0.0 (clang-1300.0

Time to build vocab: 0.04 mins


In [116]:
start = time()

w2v_model.train(sentences,
                total_examples=w2v_model.corpus_count,
                epochs=30,
                report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - start) / 60, 2)))

w2v_model.init_sims(replace=True)

INFO - 15:46:05: Word2Vec lifecycle event {'msg': 'training model with 7 workers on 70944 vocabulary and 300 features, using sg=0 hs=0 sample=1e-05 negative=20 window=4 shrink_windows=True', 'datetime': '2023-05-20T15:46:05.471098', 'gensim': '4.3.1', 'python': '3.9.9 (v3.9.9:ccb0e6a345, Nov 15 2021, 13:06:05) \n[Clang 13.0.0 (clang-1300.0.29.3)]', 'platform': 'macOS-13.3-arm64-arm-64bit', 'event': 'train'}
INFO - 15:46:06: EPOCH 0 - PROGRESS: at 31.36% examples, 269543 words/s, in_qsize 14, out_qsize 0
INFO - 15:46:07: EPOCH 0 - PROGRESS: at 66.60% examples, 285040 words/s, in_qsize 13, out_qsize 0
INFO - 15:46:08: EPOCH 0: training on 2009180 raw words (858306 effective words) took 2.9s, 298831 effective words/s
INFO - 15:46:09: EPOCH 1 - PROGRESS: at 32.88% examples, 280976 words/s, in_qsize 13, out_qsize 0
INFO - 15:46:10: EPOCH 1 - PROGRESS: at 69.13% examples, 295724 words/s, in_qsize 13, out_qsize 0
INFO - 15:46:11: EPOCH 1: training on 2009180 raw words (858758 effective words)

Time to train the model: 1.47 mins


In [117]:
w2v_model.save("word2vec.model")

INFO - 15:47:33: Word2Vec lifecycle event {'fname_or_handle': 'word2vec.model', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2023-05-20T15:47:33.636073', 'gensim': '4.3.1', 'python': '3.9.9 (v3.9.9:ccb0e6a345, Nov 15 2021, 13:06:05) \n[Clang 13.0.0 (clang-1300.0.29.3)]', 'platform': 'macOS-13.3-arm64-arm-64bit', 'event': 'saving'}
INFO - 15:47:33: storing np array 'vectors' to word2vec.model.wv.vectors.npy
INFO - 15:47:33: storing np array 'syn1neg' to word2vec.model.syn1neg.npy
INFO - 15:47:33: not storing attribute cum_table
INFO - 15:47:33: saved word2vec.model


In [119]:
from sklearn.cluster import KMeans
word_vectors = Word2Vec.load("word2vec.model").wv

model = KMeans(n_clusters=2,
               max_iter=1000,
               random_state=True,
               n_init=50)
model.fit(X=word_vectors.vectors.astype('double'))

INFO - 15:48:00: loading Word2Vec object from word2vec.model
INFO - 15:48:01: loading wv recursively from word2vec.model.wv.* with mmap=None
INFO - 15:48:01: loading vectors from word2vec.model.wv.vectors.npy with mmap=None
INFO - 15:48:01: loading syn1neg from word2vec.model.syn1neg.npy with mmap=None
INFO - 15:48:01: setting ignored attribute cum_table to None
INFO - 15:48:01: Word2Vec lifecycle event {'fname': 'word2vec.model', 'datetime': '2023-05-20T15:48:01.365443', 'gensim': '4.3.1', 'python': '3.9.9 (v3.9.9:ccb0e6a345, Nov 15 2021, 13:06:05) \n[Clang 13.0.0 (clang-1300.0.29.3)]', 'platform': 'macOS-13.3-arm64-arm-64bit', 'event': 'loaded'}


KMeans(max_iter=1000, n_clusters=2, n_init=50, random_state=True)

In [120]:
word_vectors.similar_by_vector(model.cluster_centers_[1], topn=10, restrict_vocab=None)

[('parma_violet', 0.9978695511817932),
 ('needles_moment', 0.997712254524231),
 ('20_ringing', 0.9977063536643982),
 ('sugar_absinthe', 0.9971377849578857),
 ('candy_wood', 0.9970799088478088),
 ('booked_capital', 0.9970647692680359),
 ('adds_dimension', 0.996945321559906),
 ('tortoiseshell_glasses', 0.996891975402832),
 ('sawed_branch', 0.996888279914856),
 ('carrot_ambrette', 0.9968278408050537)]

In [121]:
positive_cluster_index = 1
positive_cluster_center = model.cluster_centers_[positive_cluster_index]
negative_cluster_center = model.cluster_centers_[1-positive_cluster_index]

In [123]:
words = pd.DataFrame(word_vectors.index_to_key)
words.columns = ['words']
words['vectors'] = words.words.apply(lambda x: word_vectors[f'{x}'])
words['cluster'] = words.vectors.apply(lambda x: model.predict([np.array(x)]))
words.cluster = words.cluster.apply(lambda x: x[0])
words['cluster_value'] = [1 if i==positive_cluster_index else -1 for i in words.cluster]
words['closeness_score'] = words.apply(lambda x: 1/(model.transform([x.vectors]).min()), axis=1)
words['sentiment_coeff'] = words.closeness_score * words.cluster_value

In [124]:
words.head(10)

,words,vectors,cluster,cluster_value,closeness_score,sentiment_coeff
0,musk,"[-0.018240415, 0.11474369, 0.06485452, 0.06379...",0,-1,0.985885,-0.985885
1,-,"[-0.07806011, 0.0885233, 0.023241187, -0.01320...",1,1,0.841872,0.841872
2,elon_musk,"[0.092020035, 0.039484542, 0.04520107, 0.04053...",0,-1,1.078684,-1.078684
3,people,"[0.0500488, 0.090776466, -0.027416097, 0.05665...",0,-1,1.372970,-1.372970
4,like,"[0.031427264, 0.14492418, 0.018667255, 0.05746...",0,-1,1.332847,-1.332847
5,would,"[0.017006392, 0.089235775, 0.04305824, 0.11443...",0,-1,1.397296,-1.397296
6,twitter,"[-0.017191386, 0.037429247, 0.07227137, 0.0013...",0,-1,1.055537,-1.055537
7,:,"[0.12599944, -0.012939106, 0.060281314, -0.044...",1,1,0.907072,0.907072
8,one,"[0.055811036, 0.07972116, 0.03603393, 0.051790...",0,-1,1.392335,-1.392335
9,think,"[0.035479654, 0.103134885, -0.0034316035, 0.06...",0,-1,1.438220,-1.438220


In [125]:
words[['words', 'sentiment_coeff']].to_csv('sentiment_dictionary.csv', index=False)